# ETL Pipeline: Analyzing Sleep Deprivation and Cognitive Performance
This script automates an **ETL pipeline** to process a dataset analyzing how sleep deprivation affects cognitive performance and emotional stability. It extracts raw participant data (including sleep metrics, cognitive test results, and demographics), transforms it by cleaning inconsistencies, normalizing numerical features, encoding categorical variables, and engineering new features (e.g., sleep efficiency). The transformed data is loaded into an SQLite database for secure storage and efficient querying, enabling robust analysis of sleep-cognition relationships. Built with `pandas`, `numpy`, and `sqlite3`, it includes error handling for reliability.

In [136]:
import pandas as pd
import numpy as np
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter
import sqlite3

## Data Extraction and Initial Loading

In [139]:
# Set the path to the file you'd like to load
file_path = "sleep_deprivation_dataset_detailed.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "sacramentotechnology/sleep-deprivation-and-cognitive-performance",
  file_path,
)

In [141]:
# Create a copy of the DataFrame
df_copy = df.copy()

## Data Exploration and Initial Cleaning

In [144]:
# Display the first few rows of the dataset
df.head()

,Participant_ID,Sleep_Hours,Sleep_Quality_Score,Daytime_Sleepiness,Stroop_Task_Reaction_Time,N_Back_Accuracy,Emotion_Regulation_Score,PVT_Reaction_Time,Age,Gender,BMI,Caffeine_Intake,Physical_Activity_Level,Stress_Level
0,P1,5.25,15,12,1.60,64.20,12,365.85,35,Female,30.53,2,1,33
1,P2,8.70,12,14,2.54,65.27,21,288.95,20,Male,27.28,3,8,37
2,P3,7.39,17,10,3.40,74.28,35,325.93,18,Male,30.00,1,2,32
3,P4,6.59,14,3,3.54,72.42,25,276.86,18,Male,34.47,5,0,23
4,P5,3.94,20,12,3.09,99.72,60,383.45,36,Male,29.70,3,4,14


In [146]:
# Display the last few rows of the dataset
df.tail()

,Participant_ID,Sleep_Hours,Sleep_Quality_Score,Daytime_Sleepiness,Stroop_Task_Reaction_Time,N_Back_Accuracy,Emotion_Regulation_Score,PVT_Reaction_Time,Age,Gender,BMI,Caffeine_Intake,Physical_Activity_Level,Stress_Level
55,P56,8.53,16,16,3.51,64.76,41,391.48,34,Male,23.01,2,5,8
56,P57,3.53,19,16,3.24,66.13,36,355.01,24,Female,34.93,2,1,25
57,P58,4.18,3,1,2.62,92.43,29,397.13,41,Male,29.85,0,1,21
58,P59,3.27,4,1,4.32,56.83,33,330.70,40,Female,24.53,4,1,29
59,P60,4.95,6,21,4.42,85.45,21,419.01,22,Male,30.53,0,10,16


In [148]:
# Check the dimensions of the dataset
df.shape

(60, 14)

In [150]:
# Get column names and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Participant_ID             60 non-null     object 
 1   Sleep_Hours                60 non-null     float64
 2   Sleep_Quality_Score        60 non-null     int64  
 3   Daytime_Sleepiness         60 non-null     int64  
 4   Stroop_Task_Reaction_Time  60 non-null     float64
 5   N_Back_Accuracy            60 non-null     float64
 6   Emotion_Regulation_Score   60 non-null     int64  
 7   PVT_Reaction_Time          60 non-null     float64
 8   Age                        60 non-null     int64  
 9   Gender                     60 non-null     object 
 10  BMI                        60 non-null     float64
 11  Caffeine_Intake            60 non-null     int64  
 12  Physical_Activity_Level    60 non-null     int64  
 13  Stress_Level               60 non-null     int64  
d

In [152]:
# Check for missing values
df.isnull().sum() 

Participant_ID               0
Sleep_Hours                  0
Sleep_Quality_Score          0
Daytime_Sleepiness           0
Stroop_Task_Reaction_Time    0
N_Back_Accuracy              0
Emotion_Regulation_Score     0
PVT_Reaction_Time            0
Age                          0
Gender                       0
BMI                          0
Caffeine_Intake              0
Physical_Activity_Level      0
Stress_Level                 0
dtype: int64

In [154]:
# Generate descriptive statistics for numerical columns
df.describe()

,Sleep_Hours,Sleep_Quality_Score,Daytime_Sleepiness,Stroop_Task_Reaction_Time,N_Back_Accuracy,Emotion_Regulation_Score,PVT_Reaction_Time,Age,BMI,Caffeine_Intake,Physical_Activity_Level,Stress_Level
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.0000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,5.805500,8.316667,12.000000,3.245000,75.005833,38.1500,332.539000,29.516667,27.329333,2.383333,4.116667,17.866667
std,1.832357,5.637320,7.577845,0.827368,13.671457,17.1334,87.800288,8.168753,4.541382,1.688362,3.108717,11.015346
min,3.120000,0.000000,0.000000,1.600000,50.900000,10.0000,201.560000,18.000000,18.740000,0.000000,0.000000,0.000000
25%,4.107500,4.000000,6.000000,2.595000,64.620000,25.0000,257.535000,21.750000,23.580000,1.000000,1.000000,8.750000
50%,5.690000,8.000000,11.500000,3.260000,74.270000,37.0000,327.210000,28.500000,27.365000,2.500000,4.000000,17.500000
75%,7.285000,13.000000,19.000000,3.965000,85.600000,54.2500,402.845000,36.000000,30.747500,4.000000,6.000000,26.250000
max,8.820000,20.000000,24.000000,4.490000,99.730000,67.0000,494.550000,43.000000,34.930000,5.000000,10.000000,40.000000


In [156]:
# Check for duplicates in the dataset
df.duplicated().sum()

0

In [158]:
# Check the data types of all columns
df.dtypes

Participant_ID                object
Sleep_Hours                  float64
Sleep_Quality_Score            int64
Daytime_Sleepiness             int64
Stroop_Task_Reaction_Time    float64
N_Back_Accuracy              float64
Emotion_Regulation_Score       int64
PVT_Reaction_Time            float64
Age                            int64
Gender                        object
BMI                          float64
Caffeine_Intake                int64
Physical_Activity_Level        int64
Stress_Level                   int64
dtype: object

## Data Transformation and Feature Engineering

In [161]:
# Convert 'Gender' to categorical type for efficient storage and analysis
df['Gender'] = df['Gender'].astype('category')

In [163]:
# Normalize 'Sleep_Hours' to a 0-1 scale for consistent comparison with other features
df['Sleep_Hours'] = (df['Sleep_Hours'] - df['Sleep_Hours'].min()) / (df['Sleep_Hours'].max() - df['Sleep_Hours'].min())

In [165]:
# One-hot encode 'Gender' to convert categorical data into numerical format for analysis
df = pd.get_dummies(df, columns=['Gender'], drop_first=True) 

In [167]:
# Create 'Sleep_Efficiency' feature to measure sleep quality relative to sleep duration
df['Sleep_Efficiency'] = df['Sleep_Quality_Score'] / df['Sleep_Hours']
df['Sleep_Efficiency'] = df['Sleep_Efficiency'].replace(np.inf, 0)

In [169]:
# Round all numerical columns to 2 decimal places for cleaner and more readable data
df = df.round(2)

In [171]:
# Rename columns for better readability and consistency in analysis
df.rename(columns={'Stroop_Task_Reaction_Time': 'Stroop_Reaction_Time', 'PVT_Reaction_Time': 'PVT_Time'}, inplace=True)

In [173]:
# Sort the dataset by 'Age' to organize participants for better analysis
df.sort_values(by='Age', inplace=True) 

In [175]:
# Reset the index to ensure a clean and consistent DataFrame after transformations
df.reset_index(drop=True, inplace=True) 

In [179]:
df.head()

,Participant_ID,Sleep_Hours,Sleep_Quality_Score,Daytime_Sleepiness,Stroop_Reaction_Time,N_Back_Accuracy,Emotion_Regulation_Score,PVT_Time,Age,BMI,Caffeine_Intake,Physical_Activity_Level,Stress_Level,Gender_Male,Sleep_Efficiency
0,P3,0.75,17,10,3.40,74.28,35,325.93,18,30.00,1,2,32,True,22.69
1,P4,0.61,14,3,3.54,72.42,25,276.86,18,34.47,5,0,23,True,23.00
2,P10,0.72,0,9,2.61,68.32,23,221.28,18,21.78,3,4,3,True,0.00
3,P11,0.00,6,12,2.23,87.21,37,319.04,18,24.74,1,5,9,True,0.00
4,P30,0.03,11,6,3.05,88.43,32,489.58,19,22.31,4,1,39,False,391.88


In [181]:
# Save the transformed data to a new CSV file for verification and further use
df.to_csv('transformed_sleep_data.csv', index=False)

## Loading Transformed Data into SQLite Database

In [216]:
# Create the 'sleep_data' table in the SQLite database

db_name = "sleep_study.db"

try:
    conn = sqlite3.connect(db_name, isolation_level="EXCLUSIVE")
    cursor = conn.cursor()
    
    create_table_query = """
    CREATE TABLE IF NOT EXISTS sleep_data (
        Participant_ID TEXT PRIMARY KEY,  -- Unique identifier for each participant
        Sleep_Hours REAL,                -- Number of hours slept (float)
        Sleep_Quality_Score INTEGER,     -- Sleep quality score (integer)
        Daytime_Sleepiness INTEGER,      -- Daytime sleepiness score (integer)
        Stroop_Reaction_Time REAL,       -- Reaction time in Stroop task (float)
        N_Back_Accuracy REAL,            -- Accuracy in N-Back test (float)
        Emotion_Regulation_Score INTEGER, -- Emotional regulation score (integer)
        PVT_Time REAL,                  -- Reaction time in PVT task (float)
        Age INTEGER,                    -- Age of participant (integer)
        BMI REAL,                       -- Body Mass Index (float)
        Caffeine_Intake INTEGER,         -- Caffeine intake level (integer)
        Physical_Activity_Level INTEGER, -- Physical activity level (integer)
        Stress_Level INTEGER,            -- Stress level (integer)
        Gender_Male INTEGER,             -- Gender (1 for Male, 0 for Female, stored as integer for SQLite compatibility)
        Sleep_Efficiency REAL            -- Sleep efficiency score (float)
    );
    """
    
    cursor.execute(create_table_query)
    conn.commit()
    print("Table created successfully")
    
except sqlite3.Error as e:
    print(f"Error creating table: {e}")
    
finally:
    if conn:
        conn.close()

Table created successfully


In [218]:
# Insert DataFrame into the SQLite database  
try:
    conn = sqlite3.connect(db_name, isolation_level="EXCLUSIVE")
    df.to_sql("sleep_data", conn, if_exists="append", index=False)
    print("Data inserted successfully")
except sqlite3.Error as e:
    print(f"Error inserting data: {e}")
finally:
    if conn:
        conn.close()

Data inserted successfully


In [220]:
# Retrieve and display all records from the 'sleep_data' table
import sqlite3

db_name = "sleep_study.db"

try:
    conn = sqlite3.connect(db_name, isolation_level="EXCLUSIVE")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM sleep_data;")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
except sqlite3.Error as e:
    print(f"Error retrieving data: {e}")
finally:
    if conn:
        conn.close()

('P3', 0.75, 17, 10, 3.4, 74.28, 35, 325.93, 18, 30.0, 1, 2, 32, 1, 22.69)
('P4', 0.61, 14, 3, 3.54, 72.42, 25, 276.86, 18, 34.47, 5, 0, 23, 1, 23.0)
('P10', 0.72, 0, 9, 2.61, 68.32, 23, 221.28, 18, 21.78, 3, 4, 3, 1, 0.0)
('P11', 0.0, 6, 12, 2.23, 87.21, 37, 319.04, 18, 24.74, 1, 5, 9, 1, 0.0)
('P30', 0.03, 11, 6, 3.05, 88.43, 32, 489.58, 19, 22.31, 4, 1, 39, 0, 391.88)
('P19', 0.43, 7, 22, 3.02, 98.94, 21, 246.51, 19, 22.02, 3, 9, 5, 1, 16.15)
('P2', 0.98, 12, 14, 2.54, 65.27, 21, 288.95, 20, 27.28, 3, 8, 37, 1, 12.26)
('P52', 0.79, 9, 16, 4.16, 60.45, 64, 271.94, 20, 29.82, 4, 2, 33, 1, 11.32)
('P50', 0.17, 14, 1, 4.07, 96.5, 33, 474.56, 20, 33.62, 0, 1, 11, 1, 80.61)
('P9', 0.61, 12, 1, 1.74, 58.94, 38, 258.28, 20, 25.24, 3, 4, 28, 0, 19.6)
('P22', 0.13, 0, 0, 2.48, 71.72, 35, 458.12, 20, 23.95, 0, 4, 10, 0, 0.0)
('P54', 0.92, 18, 16, 4.31, 67.93, 58, 254.86, 21, 20.83, 0, 5, 25, 0, 19.54)
('P45', 0.25, 0, 16, 4.15, 63.42, 65, 255.3, 21, 34.62, 4, 8, 20, 1, 0.0)
('P41', 0.11, 4, 21